In [51]:
import os
import pandas as pd
import numpy as np
import librosa
import librosa.display
import glob

In [52]:
from sklearn.preprocessing import LabelEncoder
from scipy.fftpack import fft
from scipy import signal
from scipy.io import wavfile
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.optimizers import Adam
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

In [53]:
class Opts():
    def __init__(self):
        self.train_data_path = "./working/train/"
        self.test_data_path = "./working/test/"
        self.csv_path = "./meta"
        self.data_path = "./data/"
        self.model_save_path = "./output/"
        self.model_name = "RF.h5"
        
        if not os.path.exists(self.train_data_path):
            os.makedirs(self.train_data_path)
        if not os.path.exists(self.test_data_path):
            os.makedirs(self.test_data_path)
        if not os.path.exists(self.csv_path):
            os.makedirs(self.csv_path)

In [54]:
opts = Opts()

In [55]:
def df_from_csv(csv_path):
    df = pd.read_csv(csv_path)
    return df

train_csv_path = os.path.join(opts.csv_path, "train.csv")
test_csv_path = os.path.join(opts.csv_path, "test.csv")

train_file_df, test_file_df = df_from_csv(train_csv_path), df_from_csv(test_csv_path)

In [56]:
# use mfcc as feature
def generateFeature(df, opts):
    X, y = [], []
    for idx, (file_path, label) in enumerate(zip(df['file_paths'], df['labels'])):
        s, sr = librosa.load(file_path)
        mfccs = np.mean(librosa.feature.mfcc(y=s, sr=sr, n_mfcc=200).T, axis=0)
        X.append(mfccs)
        y.append(label)
    return X, y

In [57]:
X_train, y_train = generateFeature(train_file_df, opts)
X_test, y_test = generateFeature(test_file_df, opts)

In [41]:
y_test

[7,
 9,
 7,
 8,
 3,
 3,
 2,
 0,
 8,
 7,
 0,
 7,
 9,
 7,
 0,
 7,
 9,
 8,
 7,
 5,
 2,
 5,
 3,
 7,
 9,
 2,
 1,
 2,
 7,
 8,
 0,
 4,
 8,
 2,
 4,
 2,
 9,
 3,
 8,
 2,
 2,
 8,
 0,
 2,
 1,
 5,
 5,
 7,
 8,
 0,
 9,
 7,
 7,
 6,
 3,
 7,
 8,
 9,
 3,
 9,
 9,
 3,
 8,
 7,
 4,
 7,
 1,
 9,
 3,
 7,
 8,
 4,
 4,
 6,
 8,
 3,
 7,
 0,
 4,
 3,
 9,
 0,
 7,
 8,
 5,
 5,
 0,
 7,
 3,
 6,
 1,
 5,
 9,
 0,
 4,
 6,
 8,
 8,
 4,
 9,
 0,
 3,
 2,
 3,
 5,
 1,
 7,
 2,
 4,
 7,
 5,
 4,
 8,
 7,
 0,
 8,
 3,
 8,
 2,
 9,
 9,
 0,
 7,
 2,
 0,
 9,
 3,
 8,
 7,
 4,
 1,
 4,
 7,
 8,
 4,
 1,
 0,
 6,
 1,
 7,
 4,
 9,
 0,
 9,
 3,
 8,
 8,
 2,
 3,
 0,
 8,
 4,
 0,
 9,
 2,
 5,
 7,
 2,
 7,
 8,
 7,
 9,
 0,
 3,
 7,
 1,
 3,
 9,
 1,
 0,
 8,
 2,
 9,
 0,
 0,
 8,
 6,
 2,
 3,
 1,
 5,
 2,
 4,
 2,
 0,
 3,
 7,
 8,
 7,
 7,
 2,
 6,
 5,
 1,
 9,
 1,
 5,
 0,
 6,
 7,
 8,
 6,
 8,
 8,
 2,
 0,
 4,
 7,
 9,
 0,
 1,
 4,
 8,
 2,
 7,
 3,
 7,
 8,
 4,
 0,
 4,
 1,
 0,
 4,
 6,
 9,
 0,
 8,
 8,
 0,
 9,
 1,
 9,
 0,
 7,
 7,
 4,
 5,
 8,
 7,
 9,
 7,
 6,
 9,
 6,
 1,
 3,
 2,
 7,
 8,


In [19]:
y_train

[2,
 6,
 0,
 0,
 7,
 7,
 4,
 7,
 5,
 4,
 3,
 7,
 7,
 8,
 7,
 5,
 4,
 8,
 9,
 4,
 2,
 7,
 8,
 7,
 8,
 8,
 9,
 2,
 7,
 3,
 9,
 6,
 8,
 9,
 9,
 5,
 7,
 7,
 8,
 9,
 5,
 4,
 2,
 7,
 4,
 0,
 3,
 7,
 2,
 0,
 3,
 7,
 4,
 2,
 0,
 3,
 0,
 7,
 0,
 0,
 7,
 4,
 3,
 4,
 8,
 7,
 4,
 5,
 3,
 2,
 7,
 7,
 2,
 7,
 8,
 6,
 9,
 7,
 2,
 7,
 4,
 8,
 3,
 7,
 2,
 4,
 7,
 8,
 7,
 5,
 2,
 9,
 3,
 4,
 8,
 7,
 7,
 9,
 4,
 2,
 7,
 7,
 0,
 0,
 0,
 3,
 3,
 0,
 3,
 0,
 7,
 0,
 0,
 3,
 3,
 7,
 2,
 7,
 7,
 9,
 8,
 3,
 5,
 5,
 7,
 4,
 9,
 9,
 3,
 7,
 7,
 3,
 8,
 9,
 9,
 8,
 7,
 4,
 5,
 7,
 5,
 7,
 6,
 7,
 8,
 7,
 3,
 6,
 9,
 0,
 0,
 7,
 1,
 3,
 2,
 6,
 3,
 0,
 7,
 3,
 0,
 6,
 5,
 1,
 3,
 3,
 3,
 2,
 3,
 3,
 7,
 4,
 8,
 8,
 7,
 9,
 9,
 3,
 5,
 7,
 5,
 1,
 1,
 7,
 7,
 8,
 8,
 3,
 9,
 7,
 3,
 7,
 5,
 7,
 2,
 5,
 3,
 8,
 9,
 8,
 9,
 7,
 6,
 6,
 2,
 7,
 3,
 4,
 5,
 0,
 7,
 0,
 4,
 2,
 3,
 0,
 2,
 5,
 7,
 2,
 6,
 5,
 1,
 7,
 0,
 9,
 8,
 5,
 2,
 2,
 9,
 5,
 8,
 2,
 8,
 9,
 6,
 7,
 2,
 7,
 9,
 4,
 9,
 4,
 5,
 0,
 8,
 9,
 4,
 7,


In [58]:
model = RandomForestClassifier()
model.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [50]:
model.n_classes_


10

In [65]:
y, sr = librosa.load("./SoundDetector/SoundDetector/air_cond/air_cond.wav")
mfccs = np.mean(librosa.feature.mfcc(y=y, sr=sr, n_mfcc=200).T, axis=0)
model.predict([mfccs])

array([0])

In [64]:
model.predict(X_test)

array([6, 9, 4, 4, 2, 3, 3, 1, 9, 3, 4, 3, 1, 4, 9, 2, 0, 3, 3, 1, 2, 4,
       3, 3, 3, 9, 2, 3, 4, 2, 4, 9, 2, 2, 9, 3, 8, 9, 4, 3, 2, 7, 0, 3,
       4, 2, 3, 3, 2, 4, 9, 4, 4, 6, 3, 1, 4, 5, 9, 2, 7, 2, 1, 7, 2, 2,
       1, 4, 4, 8, 2, 2, 3, 3, 3, 2, 3, 3, 2, 9, 9, 3, 9, 3, 3, 3, 2, 5,
       2, 8, 4, 4, 7, 3, 4, 3, 1, 2, 2, 4, 9, 4, 3, 4, 3, 9, 2, 4, 2, 4,
       1, 2, 6, 2, 2, 2, 4, 0, 4, 1, 3, 2, 4, 2, 3, 2, 3, 3, 3, 4, 4, 3,
       4, 4, 8, 9, 2, 3, 1, 2, 2, 0, 6, 2, 9, 2, 4, 3, 4, 4, 8, 3, 4, 1,
       3, 2, 7, 2, 2, 2, 4, 7, 3, 2, 4, 4, 9, 3, 7, 0, 1, 3, 2, 4, 3, 9,
       1, 7, 2, 2, 4, 9, 0, 7, 7, 4, 2, 4, 5, 1, 2, 9, 3, 4, 9, 3, 4, 3,
       3, 9, 2, 3, 4, 2, 2, 9, 9, 1, 9, 9, 4, 9, 2, 1, 2, 9, 4, 3, 4, 3,
       9, 3, 2, 1, 9, 2, 4, 3, 7, 5, 2, 3, 4, 8, 3, 7, 4, 3, 6, 2, 2, 5,
       2, 3, 9, 9, 9, 3, 4, 4, 2, 3, 2, 9, 2, 3, 2, 2, 2, 7, 4, 3, 3, 2,
       4, 2, 1, 3, 9, 7, 4, 9, 3, 1, 7, 8, 4, 6, 2, 4, 2, 3, 4, 7, 2, 9,
       0, 2, 2, 9, 3, 4, 0, 3, 3, 0, 2, 8, 7, 1, 2,

In [60]:
model.score(X_test, y_test)

0.29945945945945945

In [61]:
import coremltools
mlmodel = coremltools.converters.sklearn.convert(model)
mlmodel.author = "Candy Dong"
mlmodel.input_description["input"] = "an audio clip"
mlmodel.output_description["classLabel"] = "a integer value"
mlmodel.short_description = "urban sound classifier"
mlmodel.save(os.path.join(opts.model_save_path,"RF.mlmodel"))